# Data Preprocessing

Import libraries and dataframe

In [ ]:
#pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output

url = 'https://drive.google.com/file/d/1-5BQRl_X4Te3lGxKx496FXJHnkwp6mfT/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

#dir = '/content/drive/MyDrive/CS180/Project/CS180_Project_cbragunton_jtderez/IMPLEMENTATION'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/CS180/Project/CS180_Project_cbragunton_jtderez/IMPLEMENTATION


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CS180/Project/CS180_Project_cbragunton_jtderez/IMPLEMENTATION


In [ ]:
print(df.head(10))

   Global_Sales  Critic_Score  Critic_Count  User_Score  User_Count   highest  \
0         82.53          76.0          51.0         8.0       322.0  NA_Sales   
1         40.24           NaN           NaN         NaN         NaN  NA_Sales   
2         35.52          82.0          73.0         8.3       709.0  NA_Sales   
3         32.77          80.0          73.0         8.0       192.0  NA_Sales   
4         31.37           NaN           NaN         NaN         NaN  NA_Sales   
5         30.26           NaN           NaN         NaN         NaN  NA_Sales   
6         29.80          89.0          65.0         8.5       431.0  NA_Sales   
7         28.92          58.0          41.0         6.6       129.0  NA_Sales   
8         28.32          87.0          80.0         8.4       594.0  NA_Sales   
9         28.31           NaN           NaN         NaN         NaN  NA_Sales   

        lowest  Platform_2600  Platform_3DO  Platform_3DS  ...  Developer_nan  \
0     JP_Sales             

# Model 3: HistGradientBoostingClassifier

HistGradientBoostingClassifier from sklearn

In [ ]:
from os import X_OK
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

#use no_null index for filtering null values

# Split features and target
X = df.drop(['highest','lowest'], axis=1)
lo = df['lowest']
hi = df['highest']

#categorical_col = ['Platform',	'Year_of_Release',	'Genre',	'Publisher', 'Developer',	'Rating']

# Split data into train, and test sets; note that grid search implements cross fold validation

X_train, X_test, hi_train, hi_test = train_test_split(X, hi, test_size=0.2, random_state=427)
X_train, X_test, lo_train, lo_test = train_test_split(X, lo, test_size=0.2, random_state=427)



In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score


lo_HistGradientBoostingClassifier = HistGradientBoostingClassifier(random_state=427)

# Fit the model
lo_HistGradientBoostingClassifier.fit(X_train, lo_train)

# Predict on the test data
lo_pred = lo_HistGradientBoostingClassifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(lo_test, lo_pred)
print("Accuracy:", accuracy)


c:\Acads\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


Accuracy: 0.8238636363636364


In [ ]:
hi_HistGradientBoostingClassifier = HistGradientBoostingClassifier(random_state=427)

# Fit the model
hi_HistGradientBoostingClassifier.fit(X_train, hi_train)

# Predict on the test data
hi_pred = hi_HistGradientBoostingClassifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(hi_test, hi_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8409090909090909


Hyperparameter Tuning for HistGradientBoostingClassifier

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Initialize HistGradientBoostingClassifier
lo_clf = HistGradientBoostingClassifier(random_state=427)

# Define the hyperparameter distributions
param_distributions = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_iter': [300, 1000],
    'max_leaf_nodes': [31, 60],
    'max_depth': [None, 10],
    'min_samples_leaf': [1,5,30, 60],
    'l2_regularization': [0, 0.01],
    'max_bins': [30, 60, 192, 255]
}

# Initialize RandomizedSearchCV
lo_random_search = RandomizedSearchCV(
    estimator=lo_clf,
    param_distributions=param_distributions,
    n_iter=11,
    cv=3,
    scoring='f1_weighted',
    verbose=2,
    n_jobs=-1,
    random_state=427
)

# Fit RandomizedSearchCV
lo_random_search.fit(X_train, lo_train)

# Get the best hyperparameters
best_lo_params = lo_random_search.best_params_
print("Best parameters found: ", best_lo_params)

# Initialize a new model with the best parameters
best_lo_clf = HistGradientBoostingClassifier(**best_lo_params, random_state=42)

# Fit the model
best_lo_clf.fit(X_train, lo_train)

# Predict on the test data
lo_pred = best_lo_clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy)
accuracy = accuracy_score(lo_test, lo_pred)
report = classification_report(lo_test, lo_pred)
print('Lowest Region')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Fitting 3 folds for each of 11 candidates, totalling 33 fits
Best parameters found:  {'min_samples_leaf': 1, 'max_leaf_nodes': 60, 'max_iter': 1000, 'max_depth': None, 'max_bins': 30, 'learning_rate': 0.01, 'l2_regularization': 0.01}
Accuracy: 0.8409090909090909
Lowest Region
Accuracy: 0.83
Classification Report:
              precision    recall  f1-score   support

    EU_Sales       0.72      0.66      0.69       543
    JP_Sales       0.87      0.89      0.88      1698
    NA_Sales       0.85      0.88      0.86       890
 Other_Sales       0.68      0.56      0.61       213

    accuracy                           0.83      3344
   macro avg       0.78      0.75      0.76      3344
weighted avg       0.83      0.83      0.83      3344



In [ ]:
# Initialize HistGradientBoostingClassifier
hi_clf = HistGradientBoostingClassifier(random_state=427)

# Define the hyperparameter distributions
param_distributions = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_iter': [300, 1000],
    'max_leaf_nodes': [31, 60],
    'max_depth': [None, 10],
    'min_samples_leaf': [1,5,30, 60],
    'l2_regularization': [0, 0.01],
    'max_bins': [30, 60, 192, 255]
}

# Initialize RandomizedSearchCV with at most 20 models searched
hi_random_search = RandomizedSearchCV(
    estimator=hi_clf,
    param_distributions=param_distributions,
    n_iter=11,
    cv=3,
    scoring='f1_weighted',
    verbose=2,
    n_jobs=-1,
    random_state=427
)

# Fit RandomizedSearchCV
hi_random_search.fit(X_train, hi_train)

# Get the best hyperparameters
best_hi_params = hi_random_search.best_params_
print("Best parameters found: ", best_hi_params)

# Initialize a new model with the best parameters
best_hi_clf = HistGradientBoostingClassifier(**best_hi_params, random_state=427)

# Fit the model
best_hi_clf.fit(X_train, hi_train)

# Predict on the test data
hi_pred = best_hi_clf.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy)
accuracy = accuracy_score(hi_test, hi_pred)
report = classification_report(hi_test, hi_pred)
print('Lowest Region')
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)

Fitting 3 folds for each of 11 candidates, totalling 33 fits
Best parameters found:  {'min_samples_leaf': 1, 'max_leaf_nodes': 60, 'max_iter': 1000, 'max_depth': None, 'max_bins': 60, 'learning_rate': 0.01, 'l2_regularization': 0}
Accuracy: 0.8304425837320574
Lowest Region
Accuracy: 0.86
Classification Report:
              precision    recall  f1-score   support

    EU_Sales       0.80      0.62      0.70       497
    JP_Sales       0.86      0.87      0.87       765
    NA_Sales       0.88      0.92      0.90      2066
 Other_Sales       0.25      0.06      0.10        16

    accuracy                           0.86      3344
   macro avg       0.70      0.62      0.64      3344
weighted avg       0.86      0.86      0.86      3344



In [ ]:
from joblib import Parallel, delayed
import joblib

joblib.dump(best_hi_clf, 'Highest_Region_HistGradientBoostingClassifier.pkl')
joblib.dump(best_lo_clf, 'Lowest_Region_HistGradientBoostingClassifier.pkl')

['Lowest_Region_HistGradientBoostingClassifier.pkl']